## This notebook generates a bench of attributes for each experiment (session) using python classes and functions 

### This notebook is at the core of the pipeline of data processing. Do not play with it lightly inside the master folder (load_preprocess_mouse)

#### 1. Only modifiy if you are sure of what you are doing and that you are solving a bug
#### 2. If you do modify you MUST commit this modification using bitbucket
#### 3. If you want to play whis notebook (to understand it better) copy it on a toy folder distinct from the master folder
#### 4. If you want to modify this code (fix bug, improve, add attributes ...) it is recommanded  to first duplicate in a draft folder. Try to keep track of your change.
#### 5. When you are ready to commit : # clear all output, clean everything between hashtag 


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import glob
import pickle
import xmltodict 
import h5py

import six
import platform
import numpy as np
import pandas as pd

import datetime

## The lines below allow to run required notebook from the master folder
if "__file__" not in dir():
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommunNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_mouse")
    os.chdir(CommunNoteBookesPath)   
    
    %run loadMouse_documentation.ipynb
    %run loadRawSpike_documentation.ipynb
    %run plotMouse_documentation.ipynb
    
#utility function to check is something is None, or empty, or np.nan
def isNone(value):
    if value is None:
        return True
    elif isinstance(value, (float, int)):
        if np.isnan(value):
            return True
        return False
    else:
        return (not value)

if "__file__" not in dir():

    if platform.system()=='Linux':
        root="/data"
    else:
        root="/Users/davidrobbe/Documents/Data/"

    print("The path to data is %s"%root)
    
# you can manually specify root data foder between hashtag lines but leave it empty before you commit on git
##############################

##############################    

# Loading raw behavior data for mice

When different, rename files to match the name of the folder (if they start with "MOU")

#### Paths
  
- **rootFolder**: "/data"
- **mouse**: "MOUXXX" (name of the rat folder)
- **experiment**: "MOUXXX\_20XX\_XX\_..." (name of the session)
- **sessionPath**: Folder of the session   
    "/data/MOUXXX/Experiments/MOUXXX_2010_04_10_18_30" 
- **fullPath**: Folder of session + basename   
    "/data/MOUXXX/Experiments/MOUXXX_2010_04_10_18_30/MOUXXX_2010_04_10_18_30"     
    fullPath +".dat" is the path from the root to the dat file
     
#### Dictionary {trial: [list of variable length]}
  Last intertrial doesn't exist, but there will be a key with an empty list

  - **beamBreakTime**: for each trial and intertrial, times when a tick of the wheel cross the beam (photodetector)
  - **lickBreakTime**: for each trial and intertrial, times when the animal lick
  
#### Boolean
  - **hasBehavior**: True if there is data for beam break time
  - **isLickTraining**: True if session was for licking training (no beam break time, or "Training" in behav.param). If True, `hasBehavior` will be False
  - **hasOptogenetic**: True if optogenetic stimulation
  - **hasEEG**: True if there is a .eeg file or a .low.kwd file (spike data downsample)
  
  if `hasBehavior` is False, the rest of the data is not loaded
  
#### List: one value per trial

  - **distanceToRun**: distance to run for the trial to be successful (cm)
  - **maxTrialDuration**: maximum time allowed for one trial (sec)
  - **valveONTime**: time where the valve for licking is ON (ms)
 
  - **realTrials**: real index of the trials (start at 1), read in behav_param if possible
  - **trials**: index of the trials (start at 0), from keys of beamBreakTime
  
  - **durationTrial**: time when the trial end, relative to its start (last beam break if sucess, maxTrialDuration if fail)
  - **realStrartTrial**: real time where the trial started
  - **realStartInterTrial**: idem for inter trial
  - **realDistanceToRun**: real distance the animal has to run (distanceToRun is divided in tick and round up, so the actual distance to run is not the same as the one set)
 
  - **startStimulation**: start time of the stimulation relative to trial start, or None
  - **stopStimulation**: stop time of the stimulation relative to trial start, or None
  - **stimulationType**: name of the stimulation, as in behav_param
  
  - **noiseDuring**: "trial" or "inter-trial"
  
#### List: one value per trial, but no real value for the last one
  There's no intertrial for the last trial. To avoid inconsistent length, we put twice the last value.
  
  - **minInterTrialDuration**: minimum time for the inter-trial (sec)
  - **immobilityDuration**: minimum time where the animal has to stay immobile for the next trial to start (sec)
  
#### Other
  - **tickDistance**: distance in cm between two ticks on the wheel
  - **maxTimeBetweenBreak**: max time between two consecutive "beam break" (sec), aka mininum speed of the animal for a successful trial
  - **nTrial**: number of trials

#### List: one value per intertrial

  - **durationInterTrial**: time when the intertrial end, relative to its start. Maximum between (last beam break + immobility duration) and minInterTrialDuration.
  
#### Other
  - **goodTrials**: trials sucessful
  - **allBeamBreak**: list of all beam breaks in real time
  - **allLickBreak**: list of all lick break in real time
  - **xmlDict**: content of the .xml file is it exist, as a dictionary
            

  
  
## Task: actual distance run by the mice
The mouse runs on top of a wheel. Beam breaks are when a tick of the wheel crosses the beam of the photodetector. The space between two ticks is approximately 2.376 cm.

In a trial, the mouse has to run **distanceToRun** (cm), which can be converted to a number of "space between ticks":

    tickSpaceToDo=distanceToRun/2.376   (rounded up to an integer)
    
A distance of a 70cm, corresponds to 30 "spaces between ticks" (`70/2.376=29.46...=30`), so the animal should do 31 "beam break". But the code actually requires the animal to do only 30 beam break, so only 29 "spaces between ticks" ("tick space" was mistaken with "tick"). For a distance set to 70cm, the animal actually has to run `29*2.376=` 68.904 cm.  
Anyway, the distance to run is never the distance set, because we round up to an integer (30 spaces would be `30*2.376=` 71.28 cm)

In [ ]:
class MouseBaseClass(object):
    '''
    saveAsPickle=True: save data as a dictionnary in a pickle file "/Analysis/rawbehaviordata.p"
    parameters: dictionnary with data not in raw text files.
    '''
    def __init__(self, root, animal, experiment, param={}, saveAsPickle=True):
        #default parameters
        self.tickDistance = 2.375829 #cm 
        self.maxTimeBetweenBreak = 1 #sec between two "beam break time"
        self.hasBehavior = False
        self.isLickTraining = False
        self.hasOptogenetic = False
        
        #new parameters
        self.parameters = param.copy()          #copy the dict so the argument "param" is not changed
        self.__dict__.update(self.parameters) #update the class attributes with the new parameters
        #check if the path is correct
        if not self.compute_paths(root, animal, experiment):
            return
        
        #rename files if needed
        self.rename_files()             
        #xml, eeg
        self.xmlDict = self.read_xml()
        self.hasEEG = self.has_eeg()
        #read break times
        self.read_ticks()        
        #trials list
        self.trials = [t for t in sorted(self.beamBreakTime) if isinstance(t, int)]      
        self.realTrials = self.get_real_trials()
        self.nTrial = len(self.realTrials)        
        # Check whether to continue loading
        if not self.is_valid():
            return                
        # list: one value per trial
        self.read_trial_parameters()              
        #process ticks
        if self.hasEEG:
            self.process_ticks_square()
        else:
            self.process_ticks()
        #read optogenetic
        self.read_optogenetic() 
        if self.hasOptogenetic:
            self.process_optogenetic()
            self.stimulationNames = self.get_stimulation_names()
        #save .p
        if saveAsPickle:
            self.save_as_pickle()
                
    #---------------------------------------------------------     
    def compute_paths(self, root, animal, experiment):
        #clean name of folders (remove unnecessary slash or backslash)
        self.root = os.sep + root.strip(os.sep)
        self.animal = animal.strip(os.sep)
        self.experiment = experiment.strip(os.sep)
        #paths
        self.sessionPath = os.path.join(self.root, self.animal, "Experiments", self.experiment)
        self.fullPath = os.path.join(self.root, self.animal, "Experiments", self.experiment, self.experiment)  
        
        #Check if the path is correct
        if self.animal not in self.experiment:
            print("WARNING: session name (%s) does not contain animal name (%s)" %(self.animal, self.experiment))
        if not os.path.exists(self.sessionPath):
            print("STOP Loading - Path does no exists: %s" %self.sessionPath)
            return False
        return True
    
    def get_dict(self):
        return self.__dict__
    
    def save_as_pickle(self, name="rawbehaviordata.p"):
        folderPath = os.path.join(self.sessionPath, "Analysis")
        if not os.path.exists(folderPath):
            os.mkdir(folderPath)
        filePath = os.path.join(folderPath, name)
        pickle.dump(self.__dict__, open(filePath, "wb" ))

    def has_eeg(self):
        if os.path.exists(self.fullPath + ".eeg"):
            return True
        elif os.path.exists(self.fullPath + ".low.kwd"):
            return True
        else:
            return False
    #---------------------------------------------------------   
    def rename_files(self):
        '''
        If the files inside the folder have not the same basename, rename them.
        ex: "MOU087_2015_08_12-16_09" -> "MOU087_2015_08_12_16_09"
        '''
        files = glob.glob(self.sessionPath + "/*")
        for path in files:
            #file
            if os.path.isfile(path):
                filename = os.path.basename(path)
                #does not contain "MOUXXX_2014_..."
                if self.experiment not in filename:
                    #starts with "MOUXXX" (it has a wrong date)
                    if filename.startswith(self.animal):
                        # extract full extension ".behav_param", ".raw.kwd"
                        extension = filename[filename.find("."):]
                        #rename
                        newName = self.experiment + extension
                        newPath = os.path.join(self.sessionPath, newName)
                        os.rename(path, newPath)
                        print("Renamed %s to %s"%(filename, newName))
                        
    #---------------------------------------------------------                        
    def is_valid(self):
        '''
        Check if there is behavioral data 
            if not, stop loading data
        If there is also a lickbreaktime:
            check if it has the same number of trials+intertrials that in beambreaktime.
            if not, discard lickbreaktime.
        If there is a .behav_param:
            check it is has the same number of trials that in beambreaktime.
            if not, print warning. If it has less trials, stop loading data
        '''
        if len(self.beamBreakTime) > 0:
            #there is beambreaktime
            self.hasBehavior = True
        elif len(self.lickBreakTime) > 0:
            #there is a lickbreaktime but no beambreaktime: it's a training session
            self.isLickTraining = True
            self.hasBehavior = False
            print("STOP Loading: Lick training session")
            return False
        else:
            #there is no break time files
            self.hasBehavior = False
            print("STOP Loading: no beam break time")
            return False
        
        # Detect if it's a licking training (the word "Training" is in behav_param)
        if os.path.exists(self.fullPath + ".behav_param"):
            with open(self.fullPath + ".behav_param") as f:
                if "Training" in f.read():
                    self.isTraining = True
                    self.hasBehavior = False
                    print("STOP Loading: Lick training session")
                    return False
        
        #check if lick break time is correct, if not discard it
        lickNumber = len(self.lickBreakTime)
        beamNumber = len(self.beamBreakTime)
        if (lickNumber > 0) and (lickNumber != beamNumber):
            print("WARNING: number of trial lick break (%s) is not the same as in beam break (%s)"
                  %(lickNumber, beamNumber))
            print("Discard lick break times")
            self.lickBreakTime = {}
        
        # If behav_param exist,
        # Check if its number of trials match the number of trials in beam break times 
        # beamNumber is trial+intertrial
        if os.path.exists(self.fullPath + ".behav_param"):
            nTrialBehav = len(self.read_in_behav("Trial", valueType=int))
            if (beamNumber//2) != nTrialBehav:
                print("WARNING: number of trial in beam break (%s) is not the same as in behav_param (%s)"
                      %(beamNumber//2, nTrialBehav))
                #if there is less trial if behav_param, the code will crash: stop loading data
                if (beamNumber/2) > nTrialBehav:
                    print("STOP Loading: more trials in .beambreaktime than in .behav_param, data can't be processed")
                    self.hasBehavior = False
                    return False   
        return True
    
    #---------------------------------------------------------  
    def read_xml(self):
        if os.path.exists(self.fullPath + ".xml"):
            with open(self.fullPath + '.xml', "rb") as f:
                d = xmltodict.parse(f, xml_attribs=True)
                return d
        return {}
                
    def look_param_or_ask_user(self, name, valueType=str, sentence=""):
        try:
            return self.parameters[name]
        except KeyError:
            if sentence == "":
                sentence = "Enter %s (type: %s):"%(name,valueType)
            return valueType(input(sentence))     
    
    def read_trial_parameters(self):
        self.distanceToRun = [self.look_param_or_ask_user("distanceToRun", float)] * self.nTrial
        self.maxTrialDuration = [self.look_param_or_ask_user("maxTrialDuration", float)] * self.nTrial
        self.valveONTime = [self.look_param_or_ask_user("valveONTime", float)] * self.nTrial
        self.minInterTrialDuration = [self.look_param_or_ask_user("minInterTrialDuration", float)] * self.nTrial
        self.immobilityDuration = [self.look_param_or_ask_user("immobilityDuration", float)] * self.nTrial
        self.noiseDuring = ["trial"] * self.nTrial
        
    def get_real_trials(self):
        return [t+1 for t in self.trials]
    
    #beam break, lick breaks
    #--------------------------------------------------------------------------------------------------------
    def read_ticks(self):
        '''
        read the beam break and lick break
        If there is an eeg/low.kwd and correct channel numbers, data is extracted from square signals
        else it's read from the txt files .beambreaktime, .lickbreaktime
        Lickbreaktime is skipped if no file/ not valid/ no channel number specified
        '''
        if self.hasEEG:
            #read .eeg file and nChannel
            if os.path.exists(self.fullPath + ".eeg"):
                data = self.read_eeg() #returns None if impossible to read
            #or read .low.kwd file and nChannel
            elif os.path.exists(self.fullPath + ".low.kwd"):
                data = self.read_low_kwd()  #returns None if impossible to read
            else:
                data = None
            if data is None:
                self.hasEEG = False
            else:
                #read the channel numbers for beam break, lick break, trialON..
                # if beam break/trial ON are None or impossible, canRead is False
                canRead = self.read_channel_numbers()
                if canRead:
                    self.read_ticks_square(data)
                    return
                else:
                    self.hasEEG = False
        #if not hasEEG, or data is None, or not canRead
        self.beamBreakTime = self.read_break_time(".beambreaktime")
        self.lickBreakTime = self.read_break_time(".lickbreaktime")
        
    def read_break_time(self, name):
        print("ERROR: read_break_time is not implemented in the base class.")
        print("You should inherit the base class and implement this function")
    
    def process_ticks(self):
        self.process_beam_break_time()
        self.compute_realTime_break() 
        
    def compute_realTime_break(self):
        '''
        List with all breaks in cumulative time
        '''
        self.allBeamBreak = []
        self.allLickBreak = []
        for trial in self.trials:
            realStart = self.realStartTrial[trial]
            lengthTrial = self.durationTrial[trial]
            
            breaks = self.beamBreakTime[trial]
            breaksRealTime = [x + realStart for x in breaks]
            breaksInter = self.beamBreakTime[trial+0.5]
            breaksInterRealTime = [x + realStart + lengthTrial for x in breaksInter]
            self.allBeamBreak += breaksRealTime + breaksInterRealTime
            
        if len(self.lickBreakTime) > 0:
            for trial in self.trials:
                realStart = self.realStartTrial[trial]
                lengthTrial = self.durationTrial[trial]
                
                licks = self.lickBreakTime[trial]
                licksRealTime = [x + realStart for x in licks]
                licksInter = self.lickBreakTime[trial + 0.5]
                licksInterRealTime = [x + realStart + lengthTrial for x in licksInter]
                self.allLickBreak += licksRealTime + licksInterRealTime
    
    def process_beam_break_time(self):
        '''
        computes information from the beam break time dictionary:
        good/bad trials, duration and start of trial/intertrial
        '''
        self.durationTrial = []
        self.durationInterTrial = []
        self.goodTrials = []
        self.realStartTrial = []
        self.realStartInterTrial = []
        self.realDistanceToRun = []
        cumulativeTime = 0
        
        for trial in self.trials:
            # number of beam break to run the distance
            nbBreakToDo = int(np.ceil(self.distanceToRun[trial] / self.tickDistance))
            # actual distance to run (since we rounded up and count tick instead of tick space)
            self.realDistanceToRun.append(self.tickDistance * (nbBreakToDo-1))
            # actual number done
            nbBreak=len(self.beamBreakTime[trial])
           
            #good or bad trial
            badTrial = False
            if nbBreak < nbBreakToDo:
                badTrial = True
            else:
                # select last nbBreakToDo
                breakTime = self.beamBreakTime[trial][-nbBreakToDo:]
                # maximum difference in the last "nbBreakToDo" breaks
                maxDiff = max(np.diff(breakTime))
                #print(trial,maxDiff)
                # test if animal above minimum speed
                if maxDiff > self.maxTimeBetweenBreak:
                    badTrial = True
                else:
                    self.goodTrials.append(trial)
            
            #end of trial
            if badTrial:
                lengthTrial = self.maxTrialDuration[trial]
            else:
                lengthTrial = self.beamBreakTime[trial][-1]
            self.durationTrial.append(lengthTrial)
            
            #intertrial
            intertrial = trial + 0.5
            breaks = [0] + self.beamBreakTime[intertrial]
            lengthInter = breaks[-1]
            if lengthInter < (self.minInterTrialDuration[trial]-self.immobilityDuration[trial]):
                lengthInter = self.minInterTrialDuration[trial]
            else:
                lengthInter = lengthInter + self.immobilityDuration[trial]
            self.durationInterTrial.append(lengthInter)
            
            #start of trial and inter-trial in real time
            self.realStartTrial.append(cumulativeTime)
            cumulativeTime = cumulativeTime + lengthTrial
            self.realStartInterTrial.append(cumulativeTime)
            cumulativeTime = cumulativeTime + lengthInter
            
    #Optogenetic
    #--------------------------------------------------------------------------------------------------------
    def read_optogenetic(self):
        '''
        if there is optogenetic data, read it and set self.hasOptogenetic=True
        else, just return
        '''
        print("ERROR: read_optogenetic is not implemented in the base class.") 
        print("You should inherit the base class and implement this function")
            
    def process_optogenetic(self):
        '''
        compute start and stop of stimulation, and stimulation type
        '''
        self.startStimulation = [] #relative to trial start
        self.stopStimulation = []
        self.realStartStimulation = [] #relative to session start
        self.realStopStimulation = []
        self.stimulationType = []
        print("ERROR: process_optogenetic is not implemented in the base class.") 
        print("You should inherit the base class and implement this function")
                                     
    def find_time_of_N_tick(self, N, breaks):
        '''
        given a list of breaks, returns the Neme tick with no pause
        '''
        if (N is None) or (N <= 0):
            return None
        if len(breaks) < N:
            return None        
        start = 0
        while (start+N) <= len(breaks):
            maxDiff = max(np.diff(breaks[start : start+N]))
            if maxDiff < self.maxTimeBetweenBreak:
                return breaks[start + N - 1]
            else:
                start += 1
        return None
    
    def get_stimulation_names(self):
        if self.hasOptogenetic:
            uniqueNames = set(self.stimulationType)
            uniqueNames.remove("No stimulation")
            if len(uniqueNames) == 0:
                self.hasOptogenetic = False
                print("2executed")
                return "No stimulation"
            elif len(uniqueNames) == 1:
                return list(uniqueNames)[0]
            else:
                return ", ".join(uniqueNames)
        else:
            return "No stimulation"
        
    #EEG
    #---------------------------------------------------------------------------------------------
    def read_eeg_parameter(self):
        try:
            self.nChannel = self.xmlDict["parameters"]["acquisitionSystem"]["nChannels"]
        except KeyError:
            self.nChannel = self.look_param_or_ask_user("nChannel", valueType=int, sentence="")
        try: 
            self.lfpSamplingRate = self.xmlDict["parameters"]["fieldPotentials"]["lfpSamplingRate"]  
        except KeyError:
            self.lfpSamplingRate = self.look_param_or_ask_user("lfpSamplingRate", valueType=float,
                                                               sentence="Sampling rate of the .eeg file")
        self.nChannel = int(float(self.nChannel))
        self.lfpSamplingRate = float(self.lfpSamplingRate)
        
    def read_eeg(self):
        self.read_eeg_parameter()
        rawData = np.fromfile(self.fullPath + ".eeg", dtype="int16")
        nbLine = len(rawData)
        if nbLine%self.nChannel != 0:
            a = str(nbLine/float(self.nChannel))
            print("Can't read eeg: wrong number for nChannel (%s/%s=%s)" %(nbLine, self.nChannel, a))
            print("Read from .beambreaktime instead")
            return None
        else:
            eeg = rawData.reshape(nbLine//self.nChannel, self.nChannel)
            return eeg
       
    #low.kwd
    #---------------------------------------------------------------------------------------------
    def read_low_kwd(self): 
        #lfp is spike sampling rate / 16 (hard coded in spikedetect)
        self.lfpSamplingRate = 1250.0  #sr=20000, 20000/16=1250
        #if not found (in case in the futur it's not hard coded, and the parameter could be missing)
        if self.lfpSamplingRate is None:
            print("Warning: no 'filter_low= ' in prm file")
            self.lfpSamplingRate = self.look_param_or_ask_user("lfpSamplingRate", valueType=float,
                                                               sentence="Sampling rate of the .low.kwd file")
        #read .low.kwd
        with h5py.File(self.fullPath + ".low.kwd","r") as kwd:  
            data = kwd.get("/recordings/0/data")[()]
            self.nChannel = data.shape[1]
            return data
        
    #Channels
    #---------------------------------------------------------------------------------------------   
    def get_channel_number(self, name):
        '''
        numbers start at zero
        read a channel number in the parameter dict, returns None if it's not there
        a negative number means counting from the end (ex: "-1" and nChannel=36 --> number=35)
        When counting from the end, we must check that the result is not an electrophy channel
        '''
        if name in self.parameters:
            number = self.parameters[name]
        else:
            return None
        if number is not None:
            number = int(float(number))
            if (number < 0) and (self.nChannelElectro is not None):
                number = self.nChannel + number
                if number < self.nChannelElectro:
                    return None
        return number
                
    def read_channel_numbers(self): 
        '''
        read the channel numbers for 3 square signals
        returns False is a number is missing, equals to None or impossible (for beamBreak and trialON only)
        '''
        if "nChannelElectro" in self.parameters:
            self.nChannelElectro = int(self.parameters["nChannelElectro"])
        else:
            self.nChannelElectro = None
        
        self.channelTrialON = self.get_channel_number("channel_trialON")
        self.channelBeamBreak = self.get_channel_number("channel_beamBreak")
        self.channelLickBreak = self.get_channel_number("channel_lickBreak")
        
        #check the values
        names = ["trial/intertrial", "beam breaks"]
        for index, ch in enumerate([self.channelTrialON, self.channelBeamBreak]):      
            if ch is None:
                return False
            elif ch >= self.nChannel:
                print("ERROR: nChannel is %s so channel '%s' can't be %s" %(self.nChannel, names[index], ch))
                print("Can't read channels, read from .beambreaktime instead")
                return False
            
        #channel_lickBreak can be None, we can still read behavior data from eeg
        if self.channelLickBreak is not None:
            if self.channelLickBreak >= self.nChannel:
                print("WARNING: nChannel is %s so channel 'lick breaks' can't be %s" %(self.nChannel, 
                                                                                       self.channelLickBreak))
                print("Lick break will not be read")
                self.channelLickBreak = None
        return True  
                   
    # Extract data from square signals (see notebook demo_detect_square_signal for details)
    #--------------------------------------------------------------------------------------------    
    def read_ticks_square(self, eegData):
        '''
        convert the square signals from the channels to beam breaks, trialON/OF, lick breaks...
        also computes trial/intertrial start and duration
        '''
        sr = self.lfpSamplingRate
        #read beam break channel
        signal, beamBreakIndex, th = self.read_channel(eegData, self.channelBeamBreak, crossIndexType="up")
        self.allBeamBreak = [x/float(sr) for x in beamBreakIndex]
        
        #read trial/intertrial channel
        trialON, self.trialONIndex, threshTrialON = self.read_channel(eegData, self.channelTrialON, 
                                                                      crossIndexType="all")
        # If the recording starts in the middle of a trial (first cross is "down": trial to intertrial),
        # we need to remove the first incomplete trial
        trialONIndexDown = self.get_down_cross_index(trialON, threshTrialON)
        trialONIndexUp = self.get_up_cross_index(trialON, threshTrialON)
        if trialONIndexDown[0] < trialONIndexUp[0]:
            trialONIndexDown.pop(0)
            self.trialONIndex.pop(0)
            
        #number of readable trials
        nTrial = len(trialONIndexDown) - 1
            
        #convert beam breaks in dictionary form
        self.beamBreakTime = self.get_dictionary_break_times(beamBreakIndex, nTrial)  
                
        #same for lick signal if it exist TODO
        if self.channelLickBreak is not None:
            signal, lickBreakIndex, th = self.read_channel(eegData, self.channelLickBreak, crossIndexType="up")
            self.lickBreakTime = self.get_dictionary_break_times(lickBreakIndex, nTrial)
            self.allLickBreak = [x/float(sr) for x in lickBreakIndex]
        else:
            self.allLickBreak = []
            self.lickBreakTime = {}     
        
        #if the last occurence is a trial, delete it (it was cut)
        maxKey = max(self.beamBreakTime)
        if isinstance(maxKey, int):
            del self.beamBreakTime[maxKey]
            if maxKey in self.lickBreakTime:
                del self.lickBreakTime[maxKey]
        else:
            maxKey = int(maxKey-0.5)
                
        #Real start and duration. Last intertrial is of unknown duration, put 0.
        self.realStartTrial = [x/float(sr) for x in trialONIndexUp][:maxKey+1]
        self.realStartInterTrial = [x/float(sr) for x in trialONIndexDown][:maxKey+1]
        self.durationTrial = [x-y for x, y in zip(self.realStartInterTrial, self.realStartTrial)]
        self.durationInterTrial = [y-x for x, y in zip(self.realStartInterTrial[:-1], self.realStartTrial[1:])] + [0]

    def get_dictionary_break_times(self, breakIndex, nTrial):
        sr = self.lfpSamplingRate
        trial = -0.5
        breakTime = {trial: []}
        i = 0
        stop = self.trialONIndex[i]
        start = stop
        maxI = len(self.trialONIndex) - 1
        for upCrossIndex in breakIndex:
            #print(upCrossIndex,"stop",stop,"trial",trial)
            while (upCrossIndex>stop) and (i<maxI):              
                trial = trial+0.5
                i += 1
                start = stop
                if trial.is_integer():
                    trial = int(trial)  #converts "1.0" into "1"
                stop = self.trialONIndex[i]
                breakTime[trial] = []
            breakTime[trial].append((upCrossIndex-start)/float(sr))
            
        #if no running or no licking till the end, add empty trials
        while trial < nTrial:
            if isinstance(trial, float) and trial.is_integer():
                trial = int(trial)
            breakTime[trial] = []
            trial += 0.5
                    
        #remove what's before the first trial
        del breakTime[-0.5]
        return breakTime
          
    def read_channel(self, data, channel, crossIndexType="all"):
        channelData = data[:,channel]  
        threshold = self.get_threshold(channelData)
        if crossIndexType == "all":
            crossIndex = self.get_all_cross_index(channelData, threshold)
        elif crossIndexType=="down":
            crossIndex = self.get_down_cross_index(channelData, threshold)
        elif crossIndexType=="up":
            crossIndex = self.get_up_cross_index(channelData, threshold)   
        return channelData, crossIndex, threshold

    def get_threshold(self, channelData):
        sortData = np.sort(channelData)
        tup = np.mean(sortData[-100:])
        tdown = np.mean(sortData[:100])
        threshold = (tup+tdown)/2.0
        return threshold

    def get_all_cross_index(self, channelData, threshold):
        difference = channelData-threshold
        asign = np.sign(difference)
        changed = ((np.roll(asign, 1) - asign) != 0).astype(int)
        changed[0] = 0
        crossIndex = np.where(changed==1)[0]
        return list(crossIndex)

    def get_down_cross_index(self, channelData, threshold, value=-2):
        difference = channelData - threshold
        asign = np.sign(difference)
        asign[asign==0] = -1
        changed = np.append([0], np.diff(asign))
        crossIndex = np.where(changed==value)[0]
        return list(crossIndex)

    def get_up_cross_index(self, channelData, threshold):
        return self.get_down_cross_index(channelData, threshold, value=2)
     
    #---------------------------------------------------------------------------------------------       
    def process_ticks_square(self):
        '''
        compute good/bad trials from beam break time dictionary
        the start/duration are computed while reading the ticks
        '''
        self.goodTrials = []
        self.realDistanceToRun = []    
        for trial in self.trials:
            # number of beam break to run the distance
            nbBreakToDo = int(np.ceil(self.distanceToRun[trial]/self.tickDistance))
            # actual distance to run (since we rounded up and count tick instead of tick space)
            self.realDistanceToRun.append(self.tickDistance * (nbBreakToDo-1))
            # actual number done
            nbBreak = len(self.beamBreakTime[trial])

            #good or bad trial
            badTrial = False
            if nbBreak < nbBreakToDo:
                badTrial = True
            else:
                # select last nbBreakToDo
                breakTime = self.beamBreakTime[trial][-nbBreakToDo:]
                # maximum difference in the last "nbBreakToDo" breaks
                maxDiff = max(np.diff(breakTime))
                #print(trial,maxDiff)
                # test if animal above minimum speed
                if maxDiff > self.maxTimeBetweenBreak:
                    badTrial = True
                else:
                    self.goodTrials.append(trial)

### Loubna data

Trial parameters (distance to run, immobility duration...) are read in .behav_param. If not found, and if not in the dictionary `param`, the user is asked for input.

Break times are read in .beambreaktime and .lickbreaktime
  
#### List: one value per trial (optogenetic read in .behav_param)
  - **stimulationShouldHappen**: whether there was a stimulation (planned!!!!). This value has been renamed stimulationShouldHappen 

  - **stimulateAtBeginning**: whether the stimulation was at the beginning of the trial
  - **stimulateUponReward**: whether the stimulation was during the reward
  - **stimulateAfterNTicks**: whether the stimulation was after N ticks of running
  - **N**: values for N
  - **stimulateBeforeEnd**: whether the stimulation was during intertrial
  - **stimulateBeforeEnd_time**: time where the stimulation starts in seconds after start of intertrial

  - **opticalDuration**: length of the stimulation (ms)
  - **stim**: percentage of stimulation occurence

In [ ]:
class LoubnaRawBehaviorData(MouseBaseClass):
          
    def read_or_look_or_ask(self, name, nameBehav, exclude=None, valueType=str):
        '''
        read a parameter in behav param (one value per trial)
        if there is no values, or less value than nTrial:
          - look in param dict for a value, or ask the user
          - use this value for every trial
        '''
        if os.path.exists(self.fullPath + ".behav_param"):
            trialParam = self.read_in_behav(nameBehav, exclude, valueType)
            l = len(trialParam)
            if l == self.nTrial:
                return trialParam
            elif l == (self.nTrial-1):  #no value for last intertrial
                trialParam.append(trialParam[-1])
                return trialParam
            else:
                print("Found %s occurence of '%s' in .behav_param, but number of trials is %s" %(l, nameBehav,
                                                                                                 self.nTrial))
        trialParam = self.look_param_or_ask_user(name, valueType)
        trialParam = [trialParam] * self.nTrial
        return trialParam
    
    def read_trial_parameters(self):
        self.distanceToRun = self.read_or_look_or_ask("distanceToRun", "distance", valueType=float)     
        self.maxTrialDuration = self.read_or_look_or_ask("maxTrialDuration", "trial duration",exclude="inter", 
                                                         valueType=float)
        self.valveONTime = self.read_or_look_or_ask("valveONTime", "valve", valueType=float)
        self.minInterTrialDuration = self.read_or_look_or_ask("minInterTrialDuration","inter-trial duration",
                                                              valueType=float)
        self.immobilityDuration = self.read_or_look_or_ask("immobilityDuration", "immobility duration", valueType=float)
        
        self.noiseDuring = self.read_in_behav("Noise", valueType=str)
        
    
    def read_break_time(self, name):
        try:
            stringAllTime = np.genfromtxt(self.fullPath+name, dtype=str)
        except (OSError, IOError, ValueError):  #no file found or file empty
            print("Error: Can't read file %s"%(self.fullPath+name))
            return {}
        if not isinstance(stringAllTime, (list, np.ndarray)) or stringAllTime.ndim==0:
            print("Error: File %s can't be read as a list"%(self.fullPath+name))
            return {}
        
        #float numbers can have comma instead of dot
        allTime = []
        for value in stringAllTime:
            value = value.replace(",", ".")
            allTime.append(float(value))
        trial = -0.5
        breakTime = {}
        for time in allTime:
            if time == 0:
                trial += 0.5
                if trial.is_integer():
                    trial = int(trial)   #make the key "1" instead of "1.0"
                breakTime[trial] = []
            else:
                breakTime[trial].append(time)
        # last "trial" should be an intertrial (float .5)
        if isinstance(trial, int):
            del breakTime[trial]
            
        return breakTime
       
    def read_in_behav(self, paramName, exclude=None, valueType=str):
        '''
        Look for lines containing "paramName" and not containing "exclude"
        Split them by white spaces 
        example: "treadmill speed:     30.00" becomes ["treadmill","speed:","30.00"])
        Return a list of their last element, as valueType (str,float,int...)
        '''
        behav = self.fullPath+".behav_param"
        result = []
        with open(behav, "r") as f:
            for line in f:
                if paramName in line:
                    if (exclude is not None) and (exclude in line):
                        continue
                    res = line.split()[-1]
                    #integer or float: replace comma by dots
                    if valueType in [int,float]:
                        res = res.replace(",",".")                 
                    #integer: convert first to float ("0.00" -> 0.00 -> 0)
                    if valueType is int:
                        res = int(float(res))
                    #boolean "TRUE" "FALSE"
                    elif valueType is bool:
                        res = (res.lower()=="true")
                    else:
                        res = valueType(res)
                    result.append(res)
        return result
    
    #optogenetic
    #--------------------------------------------------------------------------------
    def read_optogenetic(self):
        '''
        if there is optogenetic data, read it and set self.hasOptogenetic=True
        else, just return
        '''
        
        ## Warning. Originally Stimulation happened meant stimulation is programmed. But if the behavioral condition are not met there was no stimulation
        ## from ocober 2016 David modified Labview code. There are 2 paprameters. Stimulation should happen and stimulation occured 
        
        ### for old files:
        self.stimulationOccured=[True]*self.nTrial
        
        if os.path.exists(self.fullPath + ".behav_param"):
            
            ### for old files:
            self.stimulationOccured=[True]*self.nTrial
            
            self.stimulationShouldHappen = self.read_in_behav("Stimulation Happened", valueType=bool)
            if len(self.stimulationShouldHappen)==0:
                self.stimulationShouldHappen = self.read_in_behav("Stimulation should happen", valueType=bool)
                self.stimulationOccured= self.read_in_behav("Stimulation Occured", valueType=bool)                

        else:
            return
        #it at least one stimulation happened the code will look for the type of stimulation
        # in some cases StimulationHppened is true but all the type of stimulation were false (Lazy Loubna ...) 
        # if all the type of stimulation are false accross al trial then hasOpto is False


        
        AllStimulationTypesAccrossTrials=[]
        if np.any(self.stimulationShouldHappen):
            self.hasOptogenetic = True
            self.stimulateAtBeginning = self.read_in_behav("at begining", valueType=bool)
            AllStimulationTypesAccrossTrials.extend(self.stimulateAtBeginning)
            self.stimulateUponReward = self.read_in_behav("upon reward", valueType=bool)
            AllStimulationTypesAccrossTrials.extend(self.stimulateUponReward)
            self.stimulateAfterNTicks = self.read_in_behav("after N ticks", valueType=bool)
            AllStimulationTypesAccrossTrials.extend(self.stimulateAfterNTicks)
            self.N = self.read_in_behav("N:", valueType=int)
            self.opticalDuration = self.read_in_behav("Optical", valueType=float)
            self.stim = self.read_in_behav("Stim Occurence", valueType=float)
            self.stimulationTime = self.read_in_behav("Stimulation time (s)", valueType=float)
            
            
            self.stimulateTrialWhenImmobile=self.read_in_behav("in trial when",valueType=bool)
            # in early experiments this type of stim does not exist in the param file
            if len(self.stimulateTrialWhenImmobile)==0:
                #create false array
                self.stimulateTrialWhenImmobile=[False]*self.nTrial
                
            AllStimulationTypesAccrossTrials.extend(self.stimulateTrialWhenImmobile)
            
            self.stimulateInterTrialWhenImmobile=self.read_in_behav("inter-trial when",valueType=bool)
      
            # in early experiments this type of stim does not exist in the param file
            if len(self.stimulateInterTrialWhenImmobile)==0:
                #create false array
                self.stimulateInterTrialWhenImmobile=[False]*self.nTrial
            
            AllStimulationTypesAccrossTrials.extend(self.stimulateInterTrialWhenImmobile)
            
            

                
            L1 = self.read_in_behav("more than",valueType=int)
            L2 = self.read_in_behav("after (s):",valueType=int)
            if np.sum(self.stimulateTrialWhenImmobile)>0:
                self.moreThan = L1[0::2]
                self.after = L2[0::2]
            elif np.sum(self.stimulateInterTrialWhenImmobile)>0:
                self.moreThan = L1[1::2]
                self.after = L2[1::2]
            else:
                self.moreThan =0
                self.after=0
            # "Stimulate before end of inter-trial": read last value of the line. This type of stim has been deleted since octobre 2016
            
            self.stimulateBeforeEnd = self.read_in_behav("before end", valueType=str)
            # in old files: "..inter-trial: FALSE". 
            
            if len(self.stimulateBeforeEnd)>0: #old files
            
                self.stimulateBeforeEnd_time = []

                if self.stimulateBeforeEnd[0] in ["TRUE", "FALSE"]:
                    self.stimulateBeforeEnd = self.read_in_behav("before end", valueType=bool)

                    if sum(self.stimulateBeforeEnd) > 0: #some are True

                        s="Enter stimulation time for `before end of inter-trial`, in seconds relative to inter-trial start"
                        #ask for missing value
                        missingTimeStim = self.look_param_or_ask_user("timeForStimulateBeforeEnd", valueType=float,
                                                                      sentence=s)     
                        self.stimulateBeforeEnd_time = [missingTimeStim if x else 0 for x in self.stimulateBeforeEnd]
                    else:
                        self.stimulateBeforeEnd_time=[False]*self.nTrial


                # in newer files: "..inter-trial: FALSE 0.0"  (float=time in seconds)  
                else:
                    self.stimulateBeforeEnd_time = self.read_in_behav("before end", valueType=float)
                    self.stimulateBeforeEnd = []
                    with open(self.fullPath+".behav_param") as f:
                        for line in f:
                            if "before end" in line:
                                if "TRUE" in line:
                                    self.stimulateBeforeEnd.append(True)
                                else:
                                    self.stimulateBeforeEnd.append(False)

                                
            else:
                #create false array to avoid error
                self.stimulateBeforeEnd=[False]*self.nTrial
            
            AllStimulationTypesAccrossTrials.extend(self.stimulateBeforeEnd)
            
            if not np.any(AllStimulationTypesAccrossTrials):
                self.hasOptogenetic = False
                                
    def process_optogenetic(self):
        '''
        compute start and stop of stimulation, and stimulation type
        '''
        
        self.startStimulation = [] #relative to trial start
        self.stopStimulation = []
        self.realStartStimulation = [] #relative to session start
        self.realStopStimulation = []
        self.stimulationType = []
        n = None
        inTrial = False
        inInterTrial = False
        for trial in self.trials:
            noStim = False
            StimCouldHavehappenedIndependtlyOfBehavior=False # there is case were occurence is above 0% but all the stim types are off
            if not self.stimulationShouldHappen[trial]:
                noStim = True
            elif self.stimulateBeforeEnd[trial]:
                # Stimulate before end of inter-trial
                # stimulateBeforeEnd_time is in seconds relative to inter-trial start
                StimCouldHavehappenedIndependtlyOfBehavior=True
                time = self.stimulateBeforeEnd_time[trial]
                start = self.durationTrial[trial] + time
                self.stimulationType.append("Stimulate before end of inter-trial (%ss)" %str(time))
            elif self.stimulateAtBeginning[trial]:
                # Stimulate at beginning of trial
                StimCouldHavehappenedIndependtlyOfBehavior=True
                start = 0
                self.stimulationType.append("Stimulate at begining of trial")
            elif self.stimulateUponReward[trial]:
                # Stimulate at end of trial (reward)
                StimCouldHavehappenedIndependtlyOfBehavior=True
                start = self.durationTrial[trial]
                self.stimulationType.append("Stimulate  upon reward")
            elif self.stimulateAfterNTicks[trial]:
                # Stimulate during trial after the Neme tick with no pause 
                StimCouldHavehappenedIndependtlyOfBehavior=True
                start = self.find_time_of_N_tick(self.N[trial], self.beamBreakTime[trial])
                if start is None:
                    noStim = True
                else:
                    n = self.N[trial]
                    self.stimulationType.append("Stimulate after %s ticks in trial" %self.N[trial])
                    
            elif self.stimulateTrialWhenImmobile[trial]:
                 # Stimulate during trial after X time and Y immo 
                StimCouldHavehappenedIndependtlyOfBehavior=True
                WheelDetectionTimes=self.beamBreakTime[trial].copy()
                WheelDetectionTimes.extend([float(self.after[trial]),self.durationTrial[trial]])
                WheelDetectionTimes=sorted(np.unique(WheelDetectionTimes))
                WheelDetectionTimesForImmoDetection=[X for X in WheelDetectionTimes if X>=self.after[trial]]
                ImmoDurations=np.diff(WheelDetectionTimesForImmoDetection)
                IndexesBigDiff=[n for n,x in enumerate(ImmoDurations) if x >= self.moreThan[trial]]    
                
                if len(IndexesBigDiff)==0:
                    noStim = True
                else:
                    inTrial = True
                    start=WheelDetectionTimesForImmoDetection[IndexesBigDiff[0]]+self.moreThan[trial]
                    self.stimulationType.append("Stimulate in trial when animal is still")
            
            elif self.stimulateInterTrialWhenImmobile[trial]:
                # Stimulate during trial after X time and Y immo
                StimCouldHavehappenedIndependtlyOfBehavior=True
                WheelDetectionTimes=self.beamBreakTime[trial+0.5].copy()
                WheelDetectionTimes.extend([float(self.after[trial]),self.durationInterTrial[trial]])
                WheelDetectionTimes=sorted(np.unique(WheelDetectionTimes))
                WheelDetectionTimesForImmoDetection=[X for X in WheelDetectionTimes if X>=self.after[trial]]
                ImmoDurations=np.diff(WheelDetectionTimesForImmoDetection)
                IndexesBigDiff=[n for n,x in enumerate(ImmoDurations) if x > self.moreThan[trial]]                
                
                if len(IndexesBigDiff)==0:
                    noStim = True
                else:
                    #start is stim start relative to trial start
                    start=WheelDetectionTimesForImmoDetection[IndexesBigDiff[0]]+self.moreThan[trial]+self.durationTrial[trial]
                    #start=time[stimIndex]+self.moreThan[-1]+self.durationTrial[trial]
        
                    #if stimulation time fall in the last 2s of immo then there is no stim
                    if start>(self.durationTrial[trial]+self.minInterTrialDuration[trial]-self.immobilityDuration[trial]):
                        noStim = True
                        start=None
                       
                    else:
                        inInterTrial = True
                        self.stimulationType.append("Stimulate in intertrial when animal is still")
                        
                
                
                
            else:
                noStim = True
                if not StimCouldHavehappenedIndependtlyOfBehavior:
                    self.stimulationShouldHappen[trial]=False
                
            if noStim:
                start = None
                stop = None
                self.stimulationOccured[trial] = False
                self.stimulationType.append("No stimulation")
                self.startStimulation.append(None)
                self.stopStimulation.append(None)
                self.realStartStimulation.append(None)
                self.realStartStimulation.append(None)
            else: 
                stop = start + self.opticalDuration[trial]/1000.0
                self.startStimulation.append(start)
                self.realStartStimulation.append(self.realStartTrial[trial] + start)           
                self.stopStimulation.append(stop)
                self.realStartStimulation.append(self.realStartTrial[trial] + stop)
                
        
        
        # now look in all trials for ficitive time of stimmulation (to compare trial with and without stim)
        # only trial with distance to run= median(all distance to rin session) are considered
        #dav
        LastAfterXTime=float(self.after[-1:][0])
        LastMoreThan=float(self.moreThan[-1:][0])
        
        MedianDistanceToRunInSession=np.median(self.distanceToRun)
        #special case for "after N ticks"
        self.PutativeStimTimeAtNTicks = []
        if n is not None:
            for trial in self.trials:
                if self.distanceToRun[trial] != MedianDistanceToRunInSession:
                    self.PutativeStimTimeAtNTicks.append(None)
                    continue
                time = self.find_time_of_N_tick(n, self.beamBreakTime[trial])
                self.PutativeStimTimeAtNTicks.append(time)
        self.PutativeStimTimeInTrial = []
        if inTrial is not False:
            for trial in self.trials:
                if self.distanceToRun[trial] != MedianDistanceToRunInSession:
                    self.PutativeStimTimeInTrial.append(None)
                    continue
                
                WheelDetectionTimes=self.beamBreakTime[trial].copy()
                WheelDetectionTimes.extend([LastAfterXTime,self.durationTrial[trial]])
                WheelDetectionTimes=sorted(np.unique(WheelDetectionTimes))
                WheelDetectionTimesForImmoDetection=[X for X in WheelDetectionTimes if X>=LastAfterXTime]
                ImmoDurations=np.diff(WheelDetectionTimesForImmoDetection)
                IndexesBigDiff=[n for n,x in enumerate(ImmoDurations) if x > LastMoreThan]    
                if len(IndexesBigDiff)==0:
                    ts=None
                    
                else:
                    inTrial = True
                    ts=WheelDetectionTimesForImmoDetection[IndexesBigDiff[0]]+LastMoreThan                       
                self.PutativeStimTimeInTrial.append(ts)
                
        self.PutativeStimTimeInInterTrial = []
        if inInterTrial is not False:
            for trial in self.trials:
                if self.distanceToRun[trial] != MedianDistanceToRunInSession:
                    self.PutativeStimTimeInInterTrial.append(None)
                    continue
                WheelDetectionTimes=self.beamBreakTime[trial+0.5].copy()
                WheelDetectionTimes.extend([LastAfterXTime,self.durationInterTrial[trial]])
                WheelDetectionTimes=sorted(np.unique(WheelDetectionTimes))
                WheelDetectionTimesForImmoDetection=[X for X in WheelDetectionTimes if X>=LastAfterXTime]
                ImmoDurations=np.diff(WheelDetectionTimesForImmoDetection)
                IndexesBigDiff=[n for n,x in enumerate(ImmoDurations) if x > LastMoreThan]   
                if len(IndexesBigDiff)==0:
                    ts=None
                else:
                    ts=WheelDetectionTimesForImmoDetection[IndexesBigDiff[0]]+LastMoreThan+self.durationTrial[trial]
                    if ts>(self.durationTrial[trial]+self.minInterTrialDuration[trial]-self.immobilityDuration[trial]):
                        ts=None
                self.PutativeStimTimeInInterTrial.append(ts)
                    
#         #test that at least one stimulation happened 
#         ##not sure this is usefull or should be put at the start of the def?)
#         ##for now this is commneted
#         if not np.any(self.stimulationShouldHappen):
#             print("executed at trial %s" %trial)
#             ###
#             print(self.stimulationShouldHappen)
#             ###
#             self.hasOptogenetic = False
                        

### Loubna raw data demo (.beambreaktime, .behav_param)
  

In [ ]:
#run only if inside this notebook (do not execute if "%run this_notebook")
if "__file__" not in dir():
    """
    Below is an example of data path information. DO NOT CHANGE THESE 2LINES BELOW
    BUT you can put your data path information between the 2 hashtag lines after the example
    (for git tracking issue)
    """
    
    SESSION="MOU087_2015_08_17_16_04"

##############################
    
##############################
    
    ANIMAL=SESSION[0:6]
    data = LoubnaRawBehaviorData(root,ANIMAL,SESSION)
    
    if data.hasBehavior and data.isLickTraining:
        print("Lick Training session")
    elif data.hasOptogenetic:
        print("Optogenetic session")
        print(data.stimulationShouldHappen)
        print(data.stimulateAfterNTicks)
        print(data.startStimulation)
    elif data.hasBehavior: 
        print("Training session")
    else:
        print("data not valid: no break time file")


### Carola Format

#### If here is an .eeg file or .low.kwd
  - Sampling rate and nChannel are read from the xml (lfpSamplingRate), or looked in param, or asked
  - Channel number for "beam break", "lick break" and "trialON" (trial/intertrial) are looked in param or asked
  - "Lick break" channel can be `None`
  - If the channel numbers for "beam break", "trial ON" are `None` or superior to `nChannel`, the eeg is not read. We switch to the .beambreaktime
  - Idem if `nChannel` doesn't divide the number of lines in the .eeg

#### else read from .beambreaktime

The .beambreaktime is read differently
  - Consecutive zeroes should be treated as one "0 0 0" -> "0" 
  - If the trial was not sucessfull, the last tick will be "60": it's not a real tick, it's the maximum duration of the trial (`maxTrialDuration`)   
    " ... 45.0 50.2 60.4 0 " -> trial=[..., 45.0, 50.2]
  - If the inter-trial was a success, the last tick is not real, it's the length of the intertrial  
    " ... 12.2 14.2 0 1.5"  -> inter-trial=[..., 12.2] and trial=[1.5, ...]

There is no .behav_param, so a default parameter dictionary is provided




In [ ]:
class CarolaRawBehaviorData(MouseBaseClass):
    
    def __init__(self, root, animal, experiment, param={}, saveAsPickle=True):
        # default parameters
        defaultParam = {
            "distanceToRun":100,
            "maxTrialDuration": 60,
            "valveONTime":50,
            "minInterTrialDuration":15,
            "immobilityDuration":2,
            "tickDistance":2.375829, #cm 
            "maxTimeBetweenBreak":1,
        }
        # update with the one provided by user
        defaultParam.update(param)
        
        #we need those two values to read break time in txt file
        self.param_maxTrialDuration = defaultParam["maxTrialDuration"]
        self.param_minInterTrialDuration = defaultParam["minInterTrialDuration"]
        
        #call default init
        MouseBaseClass.__init__(self, root, animal, experiment, defaultParam, saveAsPickle)
        
    def is_valid(self):
        '''
        Check if there is behavioral data 
            if not, stop loading data
        If there is also a lickbreaktime:
            check if it has the same number of trials+intertrials that in beambreaktime.
            if not, discard lickbreaktime.
        If there is a .behav_param:
            check it is has the same number of trials that in beambreaktime.
            if not, print warning. If it has less trials, stop loading data
        '''
        if len(self.beamBreakTime) > 0:
            #there is beambreaktime
            self.hasBehavior = True
        elif len(self.lickBreakTime) > 0:
            #there is a lickbreaktime but no beambreaktime: it's a training session
            self.isLickTraining = True
            self.hasBehavior = False
            print("STOP Loading: Lick training session")
            return False
        else:
            #there is no break time files
            self.hasBehavior = False
            print("STOP Loading: no beam break time")
            return False
        
        #check if lick break time is correct, if not discard it
        lickNumber = len(self.lickBreakTime)
        beamNumber = len(self.beamBreakTime)
        if lickNumber!=beamNumber:
            #try to load lick break differently
            self.lickBreakTime = self.read_break_time('.lickbreaktime', cumulZero=False)
            lickNumber = len(self.lickBreakTime)
        if (lickNumber>0) and (lickNumber!=beamNumber):
            print("WARNING: number of trial lick break (%s) is not the same as in beam break (%s)"
                  %(lickNumber, beamNumber))
            print("Discard lick break times")
            self.lickBreakTime = {}
        return True
                    
    # .beamBreakTime
    #--------------------------------------------------------------------------------------------    
    def read_break_time(self, name, cumulZero=True):
        '''
        read the break time to get the same results as the other format
        carola format: 0 0 1 2 3 ... 60 0 0 ...15 0 0 (15 and 60 are not real ticks)
        if cumulZero: "0 0 0.." should be "0"
        '''
        try:
            allTime = pd.read_csv(self.fullPath+name, header=None).values[:,0]
        except (OSError, IOError, ValueError):  #no file found or file empty
            return {}
        trial = -0.5
        
        #first line should be a zero
        if allTime[0] != 0:
            allTime = np.append([0], allTime)
            
        breakTime = {}
        previousWasZero = False
        isInt = False
        for index, time in enumerate(allTime):
            if (time==0):
                if not cumulZero:
                    previousWasZero = False
                if not previousWasZero:
                    trial += 0.5
                    if trial.is_integer():
                        trial = int(trial)   #make the key "1" instead of "1.0"
                        isInt = True
                    else:
                        isInt = False
                    breakTime[trial] = []
                previousWasZero = True
            else:
                previousWasZero = False
                # "0" is never a tick
                if time == 0:
                    continue
                #in trial, "60" is not a tick
                if isInt and time >= self.param_maxTrialDuration:
                    continue
                #in intertrial, "X 0", X is not a tick (but length of intertrial).
                if (not isInt) and (time>=self.param_minInterTrialDuration):
                    #check if followed by "0" (or end of list)
                    index = index+1
                    if (index==len(allTime)) or (allTime[index]==0):
                        continue
                #if it's a tick
                breakTime[trial].append(time)
              
        # last "trial" should be an intertrial (float .5)
        if isinstance(trial, int):
            del breakTime[trial]

        return breakTime
    
    #---------------------------------------------------------------------------------------------
    def read_optogenetic(self):
        return
    

### Carola Data demo

In [ ]:
#run only if inside this notebook (do not execute if "%run this_notebook")
if "__file__" not in dir():

    
    SESSION="/MOU074_2015_07_17_12_54"


##############################

##############################
    
    ANIMAL=SESSION[0:6]
        
    #Those parameters are overwrite if there is a .behav_param file
    paramCarola={
        "distanceToRun":100,
        "maxTrialDuration": 60,
        "valveONTime":50,
        "minInterTrialDuration":15,
        "immobilityDuration":2,
        #to read .eeg (put None to not read .eeg)
        "nChannelElectro":32,
        "channel_opto": -6, #not used
        "channel_lickBreak":-5,
        "channel_reward": -4, #not used currently
        "channel_sound": -3, #not used
        "channel_trialON": -2,
        "channel_beamBreak": -1,
    }
    data=CarolaRawBehaviorData(root,ANIMAL,SESSION,paramCarola)  
    
    if data.isLickTraining:
        print("Training session")
    elif data.hasBehavior:   
        print(data.hasEEG)
        print(len(data.realStartTrial))
        print(sorted(data.beamBreakTime), len(data.beamBreakTime))
        print(sorted(data.lickBreakTime), len(data.lickBreakTime))
    else:
        print("data not valid: no break time file")


## Preprocess data

real time: time in seconds from the start of the session

Raw data are read from pickle file if it exist (and redoPreprocess=False).
Otherwise if there is a .behav_param, `LoubnaRawBehaviorData` is called, if there is not, `CarolaRawBehaviorData` is called.

  - **taskResults**: dictionary with statistics on the session
  - **totalDurationMin**: duration of the session in minutes
 

In [ ]:
class PreprocessMouse():
    def __init__(self, root, animal, experiment, param={}, saveAsPickle=True, redo=False):
        self.hasBehavior = False
        
        #paths
        if not self.compute_path(root, animal, experiment):
            return
        
        #default parameter that can be override by param
        self.binSize = 0.25
        
        #load behavior data
        behavDict = self.get_behavior_dict(param,saveAsPickle,redo)

        #update class attributes
        self.__dict__.update(param)     #first with parameter provided
        self.__dict__.update(behavDict) #next with raw data attributes
        
        #Stop if not valid or training session
        if (not self.hasBehavior) or self.isLickTraining:
            return
       
        self.taskResults = self.compute_result()
        self.totalDurationMin = (sum(self.durationTrial) + sum(self.durationInterTrial))/60.0
    
        if saveAsPickle:
            self.save_as_pickle()
         
    def compute_path(self, root, animal, experiment):
        #clean name of folders (remove unnecessary slash or backslash)
        self.root = os.sep + root.strip(os.sep)
        self.animal = animal.strip(os.sep)
        self.experiment = experiment.strip(os.sep)
        #paths
        self.sessionPath = os.path.join(self.root, self.animal, "Experiments", self.experiment)
        self.fullPath = os.path.join(self.root, self.animal, "Experiments", self.experiment, self.experiment)  
        
        #Check if the path is correct
        if self.animal not in self.experiment:
            print("WARNING: session name (%s) does not contain animal name (%s)" %(self.animal, self.experiment))
        if not os.path.exists(self.sessionPath):
            print("STOP Loading - Path does no exists: %s" %self.sessionPath)
            return False
        return True
            
    def get_dict(self):
        return self.__dict__
    
    def save_as_pickle(self, folder="Analysis", name="preprocess.p"):
        name = "preprocesseddata_binsize" + np.str(int(self.binSize*1000)) + "ms_.p"
        folderPath = os.path.join(self.sessionPath, folder)
        if not os.path.exists(folderPath):
            os.mkdir(folderPath)
        filePath = os.path.join(folderPath, name)
        pickle.dump(self.__dict__, open(filePath, "wb" ))
          
    def get_behavior_dict(self, param, saveAsPickle, redo):
        analysisPath = os.path.join(self.sessionPath, "Analysis")
        picklePath = os.path.join(analysisPath, "rawbehaviordata.p")
        # if not redo, and there is a pickle, try to read it
        if os.path.exists(picklePath) and (not redo):
            try:
                behaviorDict = pickle.load(open(picklePath, "rb"))
                print("Behavior data loaded from %s" %picklePath)
                return behaviorDict
            except:
                pass
        #otherwise, read the raw data
        if glob.glob(self.sessionPath + "/*.behav_param"):
            behaviorDict = LoubnaRawBehaviorData(self.root, self.animal, self.experiment, param, 
                                                 saveAsPickle).get_dict()
        elif glob.glob(self.sessionPath + "/*.beambreaktime"):
            behaviorDict = CarolaRawBehaviorData(self.root, self.animal, self.experiment, param,
                                                 saveAsPickle).get_dict()
        else:
            self.hasBehavior = False
            print("No behavior data")
            return {}
        print("Behavior data loaded from rawfiles")
        return behaviorDict
    
    #--------------------------------------------------------------------------------
    def compute_result(self):
        #Correct stop: length of intertrial is the minimum possible (mouse did not run a lot)
        correctStop = [trial for trial in self.goodTrials 
                     if abs(self.durationInterTrial[trial]-self.minInterTrialDuration[trial]) <= 0.25]
        
        #Percentage of correct stop
        perCorrectStop = len(correctStop)/float(self.nTrial)*100
        
        #Percentage of correct trials
        perGoodTrial = (len(self.goodTrials)/float(self.nTrial))*100
        
        #Duration of the whole session in seconds
        sessionDuration = sum(self.durationTrial) + sum(self.durationInterTrial)
        
        #Number of trial per minute
        nbTrialsPerMin = (self.nTrial/(sessionDuration/60.0))
        
        #Number of good trials per minute
        nbGoodTrialsPerMin = len(self.goodTrials)/(sessionDuration/60.0)
        
        #Mean running speed in cm/sec
        nbTicks = len(self.allBeamBreak)
        overallRunningSpeed = nbTicks*self.tickDistance/float(sessionDuration)
        
        #Mean intertrial duration after a good trial
        if len(self.goodTrials) > 0:
            goodDurationInter = [x for (trial,x) in enumerate(self.durationInterTrial) if trial in self.goodTrials]
            medianInter = np.nanmedian(goodDurationInter)
        else:
            medianInter = np.nan
        
        #hist of speed during trial
        TimeBorder=10 # (in sec) run speed will be quantified from this time before end of trial
        # to this time after the putative end of intertrial
        beamBreak = np.asarray(self.allBeamBreak)
        timeBin20 = np.arange(0,20*self.binSize, self.binSize)
        timeBin16 = np.arange(0,16*self.binSize, self.binSize)
        
        MeanDurationIntertrial=np.median(self.minInterTrialDuration)
        MeanDurationImmobility=np.median(self.immobilityDuration)
        if MeanDurationIntertrial<MeanDurationImmobility:
            MeanDurationIntertrial=MeanDurationImmobility
        
        
        timebinBeforeAndAfterTrialEnd=np.arange(-TimeBorder,MeanDurationIntertrial+TimeBorder,self.binSize)
        startHist = []
        goodEndHist = []
        interHist = []
        BeforeAndAfterTrialEndHist=[]
        for trial in self.trials:
            #start
            zero = self.realStartTrial[trial]
            alignedBreak = beamBreak-zero
            hist, bins = np.histogram(alignedBreak, timeBin20)
            startHist.append(hist)
            #good end
            if trial in self.goodTrials:
                zero = self.realStartTrial[trial] + self.durationTrial[trial]
                alignedBreak = - beamBreak + zero #aligned on bin 0, 0.25, 0.5 ...
                hist, bins = np.histogram(alignedBreak, timeBin20)
                goodEndHist.append(hist)
            #end inter trial (do not process last intertrial, it can be empty)
            if (trial+1) < self.nTrial:
                zero = self.realStartTrial[trial] + self.durationTrial[trial] + self.durationInterTrial[trial]
                alignedBreak = -beamBreak + zero
                hist,bins = np.histogram(alignedBreak, timeBin16)
                interHist.append(hist)
                
                
            # aligned relative to end of trial (for good and bad)
            zero = self.realStartTrial[trial] + self.durationTrial[trial]
            alignedBreak = beamBreak - zero
            hist, bins = np.histogram(alignedBreak, timebinBeforeAndAfterTrialEnd)
            BeforeAndAfterTrialEndHist.append(hist)
        
        
        
        uSpeed = self.tickDistance/float(self.binSize)
        MeanSpeedBeforeAndAfterTrialEnd=np.nanmean(BeforeAndAfterTrialEndHist,0)*uSpeed
        smoothedNormalizedMeanRunSpeedAlignedTrialEnd=smooth(MeanSpeedBeforeAndAfterTrialEnd,1)/smooth(MeanSpeedBeforeAndAfterTrialEnd,1).max()
        MaxValueThisRun=smoothedNormalizedMeanRunSpeedAlignedTrialEnd[0:int(TimeBorder/self.binSize)].max()
        MinValueIntertrial=smoothedNormalizedMeanRunSpeedAlignedTrialEnd[int(TimeBorder/self.binSize):int((MeanDurationIntertrial+TimeBorder)/self.binSize)].min()
        MaxValueNextRun=smoothedNormalizedMeanRunSpeedAlignedTrialEnd[-int(TimeBorder/self.binSize):].max()
        RunAndStopIndex=(MaxValueThisRun-MinValueIntertrial)*(MaxValueNextRun-MinValueIntertrial)
        
        
        MaxValueThisRun=smoothedNormalizedMeanRunSpeedAlignedTrialEnd[0:int(TimeBorder/self.binSize)].max()
        MinValueIntertrial=smoothedNormalizedMeanRunSpeedAlignedTrialEnd[int(TimeBorder/self.binSize):int((MeanDurationIntertrial+TimeBorder)/self.binSize)].min()
        MaxValueNextRun=smoothedNormalizedMeanRunSpeedAlignedTrialEnd[-int(TimeBorder/self.binSize):].max()
        
        RunAndStopIndex=(MaxValueThisRun-MinValueIntertrial)*(MaxValueNextRun-MinValueIntertrial)
        
        
        
#         RunSpeed=np.mean(MeanSpeedBeforeAndAfterTrialEnd[0:20])
#         NextRunSpeed=np.mean(MeanSpeedBeforeAndAfterTrialEnd[-20:])
#         IndexEndMinIntertrial=int(20+MeanMinInterTrialDuration/self.binSize)
#         IntertrialSpeed=np.mean(MeanSpeedBeforeAndAfterTrialEnd[20:IndexEndMinIntertrial])
        
#         # Run and Stop Quality
        
#         RunAndStopIndex=(RunSpeed-IntertrialSpeed)*(NextRunSpeed-IntertrialSpeed)
        
        
        runningAfterSound = np.nanmean(startHist) * uSpeed         #mean speed in 20 Binsize after trial start
        runningEndInterTrial = np.nanmean(interHist) * uSpeed      #mean speed in 16 binsize before end inter-trial
        if len(self.goodTrials) > 0:
            runningBeforeReward = np.nanmean(goodEndHist) * uSpeed #mean speed in 20 Binsize before reward
        else:
            runningBeforeReward = np.nan
        
        #median trial duration
        medianTrialDuration = np.nanmedian(self.durationTrial)
          
        #median intertrial duration
        medianAllInter = np.nanmedian(self.durationInterTrial)
            
        # percentage of immobile time (no tick in 2s) during minimum time of intertrial, following good trials
        IntertrialTicks = {}
        InterTrialImmobileFraction = []
        for trial in self.goodTrials:
            intertrial = trial+0.5
            IntertrialTicks[intertrial] = [0] + self.beamBreakTime[intertrial] + [self.minInterTrialDuration[trial]]
            EarlyIntertrialTicksDiff = [j-i for i,j in zip(IntertrialTicks[intertrial][:-1], IntertrialTicks[intertrial][1:]) 
                                      if i < self.minInterTrialDuration[trial]]
            m = self.minInterTrialDuration[trial]
            if m != 0:
                InterTrialImmobileFraction.append(sum([i for i in EarlyIntertrialTicksDiff if i>=2])/m*100)   
            
        InterTrialImmobileFraction = np.nanmedian(InterTrialImmobileFraction)
        
        #other
        if runningEndInterTrial != 0:
            speedRatio = runningAfterSound/float(runningEndInterTrial)
        else:
            speedRatio = np.nan
        
        wheelTaskResults = {
            'Percentage Correct Stop (%)':perCorrectStop,
            'Real distance To Run (cm)':self.realDistanceToRun[-1],
            'Percent Correct Trial (%)': perGoodTrial,
            'Trials Per 5 Min (trials/5min)':nbTrialsPerMin*5,
            'Good trials Per 5 Min (trials/5min)':nbGoodTrialsPerMin*5,
            'Median Trial Duration (s)': medianTrialDuration,
            'Median InterTrial Duration After Correct Trials (s)':medianInter,
            'Running Speed After Sound (%ss after trial start, cm/s)'%(20*self.binSize):runningAfterSound, 
            'Overall Running Speed (cm/s)': overallRunningSpeed,
            'Running Speed Before Reward (%ss before reward, cm/s)'%(20*self.binSize):runningBeforeReward,
            'Running Speed End InterTrial (%ss before end, cm/s)'%(20*self.binSize):runningEndInterTrial,
            '% correct trial x % correct stop':(perGoodTrial*perCorrectStop)/100.0,
            'Median trial + interTrial duration (s)':medianTrialDuration+medianAllInter,
            'Speed after sound/ Speed interTrial': speedRatio,
            'Immobility Fraction in interTrial':  InterTrialImmobileFraction,
            'Immobility Fraction/ Median Trial Duration' :InterTrialImmobileFraction/medianTrialDuration,
            'RunAndStopIndex':RunAndStopIndex
        }
        return wheelTaskResults
            
               

### Preprocess demo

In [ ]:
#run only if inside this notebook (do not execute if "%run this_notebook")
if "__file__" not in dir():
 
    SESSION="MOU015_2013_07_22_23_02"


##############################
    
##############################
    
    ANIMAL=SESSION[0:6]

    #Those parameters are overwrite if there is a .behav_param file
    paramCarola={
        "distanceToRun":100,
        "maxTrialDuration": 60,
        "valveONTime":50,
        "minInterTrialDuration":15,
        "immobilityDuration":2,
        #to read .eeg (put None to not read .eeg)
        "channel_sound": 33, #not used currently
        "channel_trialON":34,
        "channel_beamBreak": 35,
        "channel_lickBreak":None,
    }
    data=PreprocessMouse(root,ANIMAL,SESSION,paramCarola,redo=True)
    print(data.taskResults)

### Class Data

Loads pickle files if they exists and redoPreprocess=False.

Otherwise, calls:

  - `PreprocessMouse` for raw data
  - `Klusta_RawSpikeData` if there is a .kwik file
  - `Kluster_RawSpikeData` if there is a .clu and no .kwik
  
Compute the date of the session according to several datetime formats. Renames folders if they're not in the right format.

Methods to save in png/html for one session

In [ ]:
class Data:
    header1 = "<!DOCTYPE html>\n<html>\n<head>\n<meta charset='utf-8' />\n<title>\n"
    header2 = "</title>\n</head><body><p>\n"
    bottom = "</p></body></html>"
    
    def __init__(self, rootFolder, animal, experiment, param={}, saveAsPickle=True, redoPreprocess=False):
        #path
        if not self.compute_path(rootFolder, animal, experiment):
            self.hasBehavior = False
            return
        
        #parameters
        binSize = 0.25
        self.__dict__.update(param)
            
        #load (or do) preprocess behavior for given bin size
        dicBehavior = self.get_preprocess_behavior_dict(binSize, rootFolder, animal, experiment, param,
                                                        saveAsPickle, redoPreprocess)
        self.__dict__.update(dicBehavior)
        
        #load raw spike 
        dicSpike = self.get_spike_data_dict(rootFolder, animal, experiment, param, saveAsPickle, redoPreprocess)
        self.__dict__.update(dicSpike)
        
        #check there is at least one cluster
        if self.hasSpike:
            n = 0
            for shank in self.clusterGroup:
                for group in self.clusterGroup[shank].values():
                    n += len(group)
            if n == 0:
                self.hasSpike = False
        
        self.date = self.get_date()
        self.daySinceStart = self.get_session_day(rootFolder)
        
    #--------------------------------------------------------------------------------  
    def compute_path(self, root, animal, experiment):
        #clean name of folders (remove unnecessary slash or backslash)
        self.root = os.sep + root.strip(os.sep)
        self.animal = animal.strip(os.sep)
        self.experiment = experiment.strip(os.sep)
        #paths
        self.sessionPath = os.path.join(self.root, self.animal, "Experiments", self.experiment)
        self.fullPath = os.path.join(self.root, self.animal, "Experiments", self.experiment, self.experiment)  
        self.analysisPath = os.path.join(self.root, self.animal, "Experiments", self.experiment, "Analysis")
        #Check if the path is correct
        if self.animal not in self.experiment:
            print("WARNING: session name (%s) does not contain animal name (%s)" %(self.animal, self.experiment))
        if not os.path.exists(self.sessionPath):
            print("STOP Loading - Path does no exists: %s" %self.sessionPath)
            return False
        return True
    
    def get_date(self, experimentName=None):
        if experimentName is None:
            experimentName = self.experiment
        else:
            experimentName = os.path.basename(experimentName).strip(os.sep)
        goodFormat = "%Y_%m_%d_%H_%M"
        dateFormats = [goodFormat, "%Y-%m-%d_%H-%M-%S", "%Y_%m_%d_%H_%M_%S", "%Y_%m_%d-%H_%M", "%Y_%m_%d-%H_%M_%S"]
        for dateFormat in dateFormats:
            fullFormat = self.animal + "_" + dateFormat
            try:
                date = datetime.datetime.strptime(experimentName, fullFormat)
            except ValueError:
                continue #try next format
            return date
        print("WARNING: session %s does not match any date formats" %experimentName)
        return None
    
    def get_session_day(self, rootFolder):
        #this session is the Xe day since the first recording of this animal
        expList = glob.glob(os.path.join(rootFolder, self.animal, "Experiments/", self.animal+"*"))
        firstSession = sorted(expList)[0]
        firstDate = self.get_date(experimentName=firstSession)
        if (firstDate is None) or (self.date is None):
            return None
        firstDate = firstDate.replace(hour=0, minute=0, second=0)
        date = self.date.replace(hour=0, minute=0, second=0)
        X = (date-firstDate).days + 1
        return X
    
    #--------------------------------------------------------------------------------        
    def get_preprocess_behavior_dict(self, binSize, rootFolder, animal, experiment, param, saveAsPickle, redo):
        name = "preprocesseddata_binsize" + np.str(int(binSize*1000)) + "ms_.p"
        preprocessPath = os.path.join(self.analysisPath, name)      
        # if not redo, and there is a pickle, try to read it    
        if os.path.exists(preprocessPath) and (not redo):
            try:
                dicBehavior = pickle.load(open(preprocessPath, "rb"))
                print("Preprocess behavior data loaded from %s" %preprocessPath)
                return dicBehavior
            except:
                pass #will redo the preprocess
        #otherwise, process the raw data
        print("Preprocessing behavior data...")
        dicBehavior = PreprocessMouse(rootFolder, animal, experiment, param, saveAsPickle, redo).get_dict()
        print("Preprocessing done")
        return dicBehavior
            
    def get_spike_data_dict(self, rootFolder, rat, experiment, param, saveAsPickle, redo):
        self.hasSpike = True
        rawSpikePath = os.path.join(self.analysisPath,"rawspikedata.p")
        if os.path.exists(rawSpikePath) and (not redo):
            try:
                spikeDict = pickle.load(open(rawSpikePath, "rb"))
                print("Spike data loaded from %s" %rawSpikePath)
                return spikeDict
            except:
                pass #will redo the preprocess
        if glob.glob(self.sessionPath + os.sep + "*.kwik"):
            try:
                spikeDict = Klusta_RawSpikeData(rootFolder, rat, experiment, param, saveAsPickle).get_dict()  
                print("Spike data loaded from raw files (Klusta .kwik format)")
                return spikeDict
            except Exception as e:
                print("Spikes (klusta .kwik) can't be loaded because of error:")
                print(e.__class__.__name__ + ": " + str(e))
                
        elif glob.glob(self.sessionPath + os.sep + "*.clu*"):
            try:
                spikeDict = Kluster_RawSpikeData(rootFolder, rat, experiment, param, saveAsPickle).get_dict()
                print("Spike data loaded from raw files (Kluster .clu format)")
                return spikeDict
            except Exception as e:
                print("Spikes (Kluster .clu) can't be loaded because of error:")
                print(e.__class__.__name__ + ": " + str(e))
        else:
            print("No spike data")
        self.hasSpike = False
        return {}
  
    #--------------------------------------------------------------------------------
    def create_empty_html(self, path, name):
        if not path.endswith(".html"):
            path = path + ".html"
        with open(path, "w") as f:
            f.write(self.header1 + name + self.header2)
            f.write(self.bottom)
            
    def insert_in_html(self, path, insertList, name=None):
        if not os.path.exists(path):
            if name is None:
                name = os.path.basename(path)
            self.create_empty_html(path, name)
        if isinstance(insertList, str):
            insertList = insertList.split("\n")
            
        with open(path, "r+") as f:
            contents = [line.rstrip("\n") for line in f if line != "\n"]
            f.seek(0)
            f.truncate() 
            contents = contents[:-1] + insertList + [self.bottom]
            f.write("\n".join(contents))
            
    def remove_lines_in_html(self, path, contents):
        if not os.path.exists(path):
            return
        newLines = []
        with open(path, "r+") as f:
            for line in f:
                if (contents not in line) and (line != "\n"):
                    newLines.append(line.strip("\n"))
            f.seek(0)
            f.truncate()
            f.write("\n".join(newLines))
      
    #--------------------------------------------------------------------------------
    def plot_session_png_html(self, plotFunctionList, name=None, override=False, eraseGeneralHTML=True, **kwargs):
        if not isinstance(plotFunctionList, list):
            plotFunctionList = [plotFunctionList]
        
        if name is None:
            name = str(plotFunctionList[0].__name__)
        
        #html for the animal
        generalName = "all_" + name
        generalFolder = os.path.join(root, animal, "Analysis")
        if not os.path.exists(generalFolder):
            os.mkdir(generalFolder)
        generalPath = os.path.join(generalFolder, generalName+".html")
        
        
        #save the plots as png, create html image tag
        images = []
        for plotFunction in plotFunctionList:
            name = plotFunction.__name__ + ".png"
            path = os.path.join(self.sessionPath, name)
            #override or not
            if (not override) and os.path.exists(path):
                print("png already exists: %s" %path)
            else:
                hasPlot = plotFunction(self, **kwargs)
                if hasPlot is False:
                    continue
                plt.savefig(path)
                plt.close()
            images.append("<a href=#%s><img src='%s' alt='%s' title='%s'/></a>"%(self.experiment, path, name, name))
            self.remove_lines_in_html(generalPath, path)
                
        #insert images in general html
        self.insert_in_html(generalPath, images, generalName)
        print("Html updated: %s" %generalPath)
            
    def plot_all_clusters_png_html(self, plotFunctionList, name=None, override=False, groupList=None, **kwargs):
        #create folder for plots
        folderPath = os.path.join(self.sessionPath, "plots")
        if not os.path.exists(folderPath):
            os.mkdir(folderPath)

        if not isinstance(plotFunctionList, list):
            plotFunctionList = [plotFunctionList]
        if not isinstance(groupList, list):
            groupList = [groupList]
            
        #html for the session
        if name is None:
            name = str(plotFunctionList[0].__name__)
        htmlName = name
        htmlPath = os.path.join(self.sessionPath, htmlName + ".html")
                
        #html for the animal
        generalName = "all_" + htmlName
        generalFolder = os.path.join(self.root, self.animal, "Analysis")
        if not os.path.exists(generalFolder):
            os.mkdir(generalFolder)
        generalPath = os.path.join(generalFolder, generalName+".html")
           
        #override if needed
        if os.path.exists(htmlPath):
            if override:
                print("Override html %s" %htmlPath)
                os.remove(htmlPath)
                #remove links in general html
                self.remove_lines_in_html(generalPath, self.experiment)
            else:
                print("Html already exists: %s" %htmlPath)
                return
    
        #save the plots as png, create html image tag
        images = []
        for shank in sorted(self.clusterGroup):
            print("Shank %s"%shank)
            for group in self.clusterGroup[shank]:
                if (groupList is not None) and (group not in groupList):
                    continue
                for cluster in sorted(self.clusterGroup[shank][group]):
                    six.print_(cluster, end=" ")
                    for plotFunction in plotFunctionList:
                        name = "shank%s_cluster%s_%s.png"%(shank, cluster, plotFunction.__name__)
                        path = os.path.join(folderPath, name)
                        hasPlot = plotFunction(self, shank, cluster, group=group, **kwargs)
                        if hasPlot is False:
                            continue
                        plt.savefig(path)
                        plt.close()
                        images.append("<a href=#%s-%s-%s><img src='%s' alt='%s' title='%s'/></a>"%(self.experiment,
                                                                                 shank, cluster, path, name, name))
                print("")
                
        #insert all images in session html
        self.insert_in_html(htmlPath, images, htmlName)
        #insert a link in general html
        link = ["<a href='"+htmlPath+"'>"+self.experiment+"</a><br>"] 
        self.insert_in_html(generalPath, link, generalName)
        
        print("Html updated: %s" %htmlPath)
        print("Html updated: %s" %generalPath)
        
    #--------------------------------------------------------------------------------
    def plot_all_clusters_png_html_insinglelocation(self, plotFunctionList, CommonFolder="ALLMOU_Analysis",name=None, override=False, groupList=None, **kwargs):
        print("override value is ")
        print(override)
        #create folder for plots
        folderPath = os.path.join(self.sessionPath, "plots")
        if not os.path.exists(folderPath):
            os.mkdir(folderPath)

        if not isinstance(plotFunctionList, list):
            plotFunctionList = [plotFunctionList]
        if not isinstance(groupList, list):
            groupList = [groupList]
            
        #html for the session

        if name is None:
            name = str(plotFunctionList[0].__name__)
        htmlName = name
        htmlPath = os.path.join(self.sessionPath, htmlName + ".html")
                
        #html for the animal
        generalName = "all_" + htmlName
        generalFolder = os.path.join(self.root,CommonFolder, "Analysis")
        if not os.path.exists(generalFolder):
            os.mkdir(generalFolder)
        generalPath = os.path.join(generalFolder, generalName+".html")
           
        #override if needed
        if os.path.exists(htmlPath):
            if override:
                print("Override html %s" %htmlPath)
                os.remove(htmlPath)
                #remove links in general html
                self.remove_lines_in_html(generalPath, self.experiment)
                print("remove %s" %self.experiment)
            else:
                print("Html already exists: %s" %generalPath)                
                return
            
        if os.path.exists(generalPath):
            if override:    
                print("Override html %s" %htmlPath)
                self.remove_lines_in_html(generalPath, self.experiment)
                
        #save the plots as png, create html image tag
        images = []
        for shank in sorted(self.clusterGroup):
            print("Shank %s"%shank)
            for group in self.clusterGroup[shank]:
                if (groupList is not None) and (group not in groupList):
                    continue
                for cluster in sorted(self.clusterGroup[shank][group]):
                    six.print_(cluster, end=" ")
                    for plotFunction in plotFunctionList:
                        name = "shank%s_cluster%s_%s.png"%(shank, cluster, plotFunction.__name__)
                        path = os.path.join(folderPath, name)
                        hasPlot = plotFunction(self, shank, cluster, group=group, **kwargs)
                        if hasPlot is False:
                            continue
                        plt.savefig(path)
                        plt.close()
                        images.append("<a href=#%s-%s-%s><img src='%s' alt='%s' title='%s'/></a>"%(self.experiment,
                                                                                 shank, cluster, path, name, name))
                print("")
                
        #insert all images in session html
        self.insert_in_html(htmlPath, images, htmlName)
        #insert a link in general html
#        link = ["<a href='"+htmlPath+"'>"+self.experiment+"</a><br>"] 
#        self.insert_in_html(generalPath, link, generalName)
        self.insert_in_html(generalPath, images, generalName)
        
        print("Html updated: %s" %htmlPath)
        print("Html updated: %s" %generalPath)
     
    #--------------------------------------------------------------------------------
    def select_cluster_group(self, selectedGroups):
        for shank in self.channelGroupList:
            selectedClusters = []
            for group in selectedGroups:
                selectedClusters += self.clusterGroup[shank][group]
            self.spikeSample[shank] = {cluID:self.spikeSample[shank][cluID] for cluID in selectedClusters}
            self.spikeTime[shank] = {cluID:self.spikeTime[shank][cluID] for cluID in selectedClusters}
            self.clusterGroup[shank] = {group:self.clusterGroup[shank][group] for group in self.selectedGroups}
            
    def add_cluster_group(self, name, shankCluList):
        for (shank, clu) in shankCluList:
            if name not in self.clusterGroup[shank]:
                self.clusterGroup[shank][name] = [clu]
            elif clu not in self.clusterGroup[shank][name]:
                self.clusterGroup[shank][name].append(clu)
     
    #--------------------------------------------------------------------------------
    def describe(self):
        dic = self.__dict__
        print("Session: %s" %self.experiment)
        print("Full Path: %s" %self.fullPath)
        print("Number of trials: %s" %self.nTrial)
        sep = "-"*(28+10+30+20+5)
        print(sep)
        print("{: <28} {: <10} {: <30} {: <30}".format("**Name**","**Type**","**Content**","**Extract**"))
        print(sep)
        for (key, value) in sorted(dic.items()):
            t = type(value).__name__
            glance = ""
            if isinstance(value, (list, np.ndarray)):
                v = "length=" + str(len(value))    
                glance = "[" + " ".join(["%.2f"%x if isinstance(x, float) else str(x) for x in value])
                if len(glance) < 30:
                    glance += "]"
                else:
                    glance = glance[:27] + "..."
            elif isinstance(value, dict):
                keys = list(value.keys())
                v = "nKeys=" + str(len(keys))
                glance = "keys: " + str(keys)
                if len(glance) > 30:
                    glance = glance[:27] + "..."
            else:
                v = value
            row = [key, t, str(v), glance]
            print("{: <28} {: <10} {: <30} {:<30}".format(*row))


### Demo

In [ ]:
#run only if inside this notebook (do not execute if "%run this_notebook")
if "__file__" not in dir():
    
    SESSION="MOU025_2014_09_05_16_38/"


##############################

##############################
    
    ANIMAL=SESSION[0:6]
    
    #Those parameters are overwritten if there is a .behav_param file
    paramCarola={
        "distanceToRun":100,
        "maxTrialDuration": 60,
        "valveONTime":50,
        "minInterTrialDuration":15,
        "immobilityDuration":2,
        #to read .eeg (put None to not read .eeg)
        "nChannelElectro":32, #32
        "channel_opto": -6, #not used
        "channel_lickBreak":-5,
        "channel_reward": -4, #not used currently
        "channel_sound": -3, #not used
        "channel_trialON": -2,
        "channel_beamBreak": -1,
    }
    data=Data(root,ANIMAL,SESSION,paramCarola,redoPreprocess=True)
    print("----------------")
    data.describe()
    